# Financial Market Simulation with Agent-Based Model

This notebook demonstrates a simple financial market simulation using an agent-based model. Agents in the market interact based on predefined strategies, influencing the price over time.

## Importing Necessary Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

## Agent Definitions and Utility Functions

In [ ]:
def trade_technical(prices):
    if len(prices) < 2:
        return 0
    return 1 if prices[-1] > prices[-2] else -1

def trade_fundamental(price, fair_value):
    return 1 if price < fair_value else -1

def trade_hft(prices):
    return np.random.choice([-1, 0, 1])

def trade_passive():
    return 1

def update_agent(agent, trade_volume, price):
    agent['wealth'] -= trade_volume * price
    agent['position'] += trade_volume

def create_agents(n_agents, strategies, wealth=10000, position=0):
    agents = []
    for _ in range(n_agents):
        strategy = np.random.choice(strategies)
        agents.append({'strategy': strategy, 'wealth': wealth, 'position': position})
    return agents

## Market Simulation Function

In [ ]:
def simulate_market(steps, initial_price, fair_value, agents):
    prices = [initial_price]
    volumes = []
    
    for _ in range(steps):
        total_demand = 0
        for agent in agents:
            strategy = agent['strategy']
            if strategy == 'technical':
                trade = trade_technical(prices)
            elif strategy == 'fundamental':
                trade = trade_fundamental(prices[-1], fair_value)
            elif strategy == 'hft':
                trade = trade_hft(prices)
            elif strategy == 'passive':
                trade = trade_passive()
            else:
                trade = 0
            
            max_trade = agent['wealth'] // prices[-1]
            trade_volume = min(max_trade, abs(trade)) * np.sign(trade)
            update_agent(agent, trade_volume, prices[-1])
            total_demand += trade_volume

        price_change = 0.1 * total_demand / len(agents)
        new_price = max(0, prices[-1] + price_change)
        prices.append(new_price)
        volumes.append(abs(total_demand))

    return prices, volumes

## Running the Simulation

In [ ]:
# Parameters
n_agents = 100
strategies = ['technical', 'fundamental', 'hft', 'passive']
agents = create_agents(n_agents, strategies)
initial_price = 100
fair_value = 120
steps = 200

# Run simulation
prices, volumes = simulate_market(steps, initial_price, fair_value, agents)

## Visualizing the Results

In [ ]:
plt.figure(figsize=(12, 6))

# Plot prices
plt.subplot(2, 1, 1)
plt.plot(prices, label='Price')
plt.axhline(y=fair_value, color='r', linestyle='--', label='Fair Value')
plt.title('Market Price over Time')
plt.xlabel('Time Step')
plt.ylabel('Price')
plt.legend()

# Plot volumes
plt.subplot(2, 1, 2)
plt.bar(range(len(volumes)), volumes, color='gray', alpha=0.7, label='Volume')
plt.title('Trading Volume over Time')
plt.xlabel('Time Step')
plt.ylabel('Volume')
plt.legend()

plt.tight_layout()
plt.show()

## Final Agent Statistics

In [ ]:
final_stats = defaultdict(lambda: {'wealth': 0, 'position': 0, 'count': 0})

for agent in agents:
    strategy = agent['strategy']
    final_stats[strategy]['wealth'] += agent['wealth']
    final_stats[strategy]['position'] += agent['position']
    final_stats[strategy]['count'] += 1

print("Final Statistics by Strategy:")
for strategy, stats in final_stats.items():
    avg_wealth = stats['wealth'] / stats['count']
    avg_position = stats['position'] / stats['count']
    print(f"{strategy.capitalize()} - Avg Wealth: {avg_wealth:.2f}, Avg Position: {avg_position:.2f}")